# Imports

In [1]:
import pandas as pd, numpy as np
import scipy as sp, statsmodels as sm

import h3
import overpass

import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon
from shapely import speedups
speedups.enable()

#from geopy import distance

import seaborn as sns, matplotlib.pyplot as plt
import pydeck

import re

import warnings
warnings.simplefilter('ignore')

np.random.seed(666) #random seed

/home/lia/miniconda3/envs/h3_fun-py38/lib/python3.8/site-packages/geopandas/_compat.py:84: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.8.1-CAPI-1.13.3). Conversions between both will be slow.
  warnings.warn(


# Get your geodata!

In this case, let's say we want to study Bench Coverage in Sweden.

Why? Well, how many times have you found yourself needing a bench during your socially-isolated hikes? 

That's right. I thought so. You are welcome.

## OpenStreetmap's Data + Overpass API

I have been surprised by how **well documented** benches seem to be in OpenStreetMap.

For inspiration, check out [overpass-turbo](http://overpass-turbo.eu/) and the [overpass python wrapper](https://github.com/mvexel/overpass-api-python-wrapper/tree/master/examples).

In [2]:
api = overpass.API(debug=False, timeout=90) #90s timeout

In [3]:
area_name = 'Sverige' #https://www.openstreetmap.org/relation/52822
adm_lvl = '2' #Country is usually 2.
amenity = 'bench' #replace this with any amenity name you like

area_req=f'area[admin_level={adm_lvl}]["name"="{area_name}"];'
node_req=f'(node["amenity"="{amenity}"](area););'

query = area_req+node_req

#https://github.com/mvexel/overpass-api-python-wrapper/blob/master/overpass/api.py
result = api.Get(query, responseformat="geojson")

In [7]:
gdf_benches=gpd.GeoDataFrame.from_features(result['features'])
gdf_benches[:5]

,geometry,amenity,backrest,source,created_by,material,table,tourism,viewpoint,colour,...,fee,seasonal,image,level,location,board_type,information,openfire,geocaching,name:en
0,POINT (16.43358 56.22450),bench,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,POINT (18.03356 59.38803),bench,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,POINT (18.03227 59.38791),bench,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,POINT (18.03186 59.38861),bench,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,POINT (18.03233 59.38916),bench,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#optionally, save it so we don't abuse of overpass ;)
#gdf_benches.to_file('./.to_ignore/out/'+'gdf_benches')